<a href="https://colab.research.google.com/github/marianabll/marianabll-Movie-recommendation-App/blob/main/dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marianabell","key":"04280619252e4d557e526e8cde15df26"}'}

In [3]:
# Criando novo diretório oculto chamado "kaggle"
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Baixando o conjunto de dados
!kaggle datasets download -d tmdb/tmdb-movie-metadata

 68% 6.00M/8.89M [00:00<00:00, 60.3MB/s]
100% 8.89M/8.89M [00:00<00:00, 74.2MB/s]


In [5]:
# Listando todos os arquivos do diretório de trabalho
!ls

kaggle.json  sample_data  tmdb-movie-metadata.zip


In [6]:
# Descompactando 
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [7]:
import pandas as pd
import numpy as np
movies_df = pd.read_csv('tmdb_5000_movies.csv')
credits_df = pd.read_csv('tmdb_5000_credits.csv')

In [8]:
credits_df.rename(columns = {'movie_id' : 'id'} , inplace = True)

In [9]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [10]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   int64 
 1   title   4803 non-null   object
 2   cast    4803 non-null   object
 3   crew    4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [11]:
# Combinando as duas bases
common_df = pd.merge(movies_df , credits_df , on = 'id')

In [12]:
# Limpando campos e registros com valores null
common_df.drop(['homepage', 'title_x', 'title_y', 'production_companies'] , axis = 1 , inplace = True)
common_df.dropna(inplace = True)
common_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3959 entries, 0 to 4801
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3959 non-null   int64  
 1   genres                3959 non-null   object 
 2   id                    3959 non-null   int64  
 3   keywords              3959 non-null   object 
 4   original_language     3959 non-null   object 
 5   original_title        3959 non-null   object 
 6   overview              3959 non-null   object 
 7   popularity            3959 non-null   float64
 8   production_countries  3959 non-null   object 
 9   release_date          3959 non-null   object 
 10  revenue               3959 non-null   int64  
 11  runtime               3959 non-null   float64
 12  spoken_languages      3959 non-null   object 
 13  status                3959 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

---


# Filtragem demográfica


---



In [13]:
# Calculando a classificação ponderada
R = common_df['vote_average']
v = common_df['vote_count']
C = common_df['vote_average'].mean()
print(C)

m = common_df['vote_count'].quantile(0.9) # Só os 10% principais filmes serão recomendados
print(m)

common_df['weighted_rating'] = (R*v + C*m) / (v + m)

6.181939883809043
2104.2000000000003


In [14]:
common_df[['original_title', 'vote_average', 'vote_count', 'weighted_rating', 'popularity']].head(10)

,original_title,vote_average,vote_count,weighted_rating,popularity
0,Avatar,7.2,11800,7.045931,150.437577
1,Pirates of the Caribbean: At World's End,6.9,4500,6.671215,139.082615
2,Spectre,6.3,4466,6.262190,107.376788
3,The Dark Knight Rises,7.6,9106,7.333824,112.312950
4,John Carter,6.1,2124,6.140778,43.926995
5,Spider-Man 3,5.9,3576,6.004443,115.699814
6,Tangled,7.4,3330,6.928350,48.681969
7,Avengers: Age of Ultron,7.3,6767,7.034802,134.279229
8,Harry Potter and the Half-Blood Prince,7.4,5293,7.053512,98.885637
9,Batman v Superman: Dawn of Justice,5.7,7004,5.811339,155.790452


In [15]:
import plotly.express as px

bar_plot = px.bar(common_df.head(10).sort_values('weighted_rating'), x='weighted_rating', y='original_title', orientation='h')

bar_plot.show()

---

# Filtragem baseada em conteúdo

---


In [17]:
common_df[['original_title', 'cast', 'crew', 'keywords' , 'genres']].head(3)

,original_title,cast,crew,keywords,genres
0,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [19]:
type(common_df.loc[0].at['cast'])

str

In [20]:
# Transformando o tipo de dado para uma lista

from ast import literal_eval

features = ['cast' , 'crew', 'keywords', 'genres']

for feature in features:
  common_df[feature] = common_df[feature].apply(literal_eval)

In [21]:
type(common_df.loc[0].at['cast'])

list

In [ ]:
common_df.loc[0].at['crew']

In [23]:
# Criando nova coluna Director

def get_director(crew):
  for crew_member in crew:
    if crew_member['job']  ==  'Director':
      return crew_member['name']
    
  return np.nan

common_df['director'] = common_df['crew'].apply(get_director)

In [26]:
common_df[['original_title' , 'director']].head()

,original_title,director
0,Avatar,James Cameron
1,Pirates of the Caribbean: At World's End,Gore Verbinski
2,Spectre,Sam Mendes
3,The Dark Knight Rises,Christopher Nolan
4,John Carter,Andrew Stanton


In [ ]:
common_df.loc[0].at['cast']

In [28]:
common_df.loc[0].at['keywords']

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

In [29]:
common_df.loc[0].at['genres']

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [30]:
# Extraindo valores das colunas cast, keywords e genres que estão associados à chave 'name
# e atualizando os valores das colunas de uma lista de dicionários para uma lista de nomes

def get_name_list(column_value):
  names_list = []
  if isinstance(column_value , list):
    for element in column_value:
      names_list.append(element['name'])

  return names_list

features = ['cast' , 'keywords' , 'genres']

for feature in features:
  common_df[feature] = common_df[feature].apply(get_name_list)

In [31]:
common_df[['original_title', 'cast', 'director', 'keywords' , 'genres']].head(3)

,original_title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]"


In [32]:
# Removendo duplicidade e pré-processando os dados

def clean_data(column_value):
  modified_list = []
  modified_string = ""
  if isinstance(column_value , list):
    for element in column_value:
      modified_string = element.replace(" " , "")
      modified_list.append(modified_string.lower())

    return modified_list

  elif isinstance(column_value , str):
    modified_string = column_value.replace(" " , "")
    return modified_string.lower()

  else:
    return ''

features = ['cast' , 'keywords' , 'genres' , 'director']

for feature in features:
  common_df[feature] = common_df[feature].apply(clean_data)

In [33]:
common_df[['original_title', 'cast', 'director', 'keywords' , 'genres']].head(3)

,original_title,cast,director,keywords,genres
0,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron,"[cultureclash, future, spacewar, spacecolony, ...","[action, adventure, fantasy, sciencefiction]"
1,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...",goreverbinski,"[ocean, drugabuse, exoticisland, eastindiatrad...","[adventure, fantasy, action]"
2,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...",sammendes,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[action, adventure, crime]"


In [34]:
# Criando um novo campo que contenha todos os metadados dos filmes 

def create_soup(x):
   return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
common_df['soup'] = common_df.apply(create_soup, axis=1)

In [37]:
common_df[['original_title', 'soup']].head(3)

,original_title,soup
0,Avatar,cultureclash future spacewar spacecolony socie...
1,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,Spectre,spy basedonnovel secretagent sequel mi6 britis...


In [44]:
links_df = pd.read_csv('movies_links.csv')

print(common_df.shape)
print(links_df.shape)

(3959, 23)
(4747, 2)


In [45]:
final_df = pd.merge(common_df, links_df, on='original_title')

In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3930 entries, 0 to 3929
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 3930 non-null   int64  
 1   budget                3930 non-null   int64  
 2   genres                3930 non-null   object 
 3   id                    3930 non-null   int64  
 4   keywords              3930 non-null   object 
 5   original_language     3930 non-null   object 
 6   original_title        3930 non-null   object 
 7   overview              3930 non-null   object 
 8   popularity            3930 non-null   float64
 9   production_countries  3930 non-null   object 
 10  release_date          3930 non-null   object 
 11  revenue               3930 non-null   int64  
 12  runtime               3930 non-null   float64
 13  spoken_languages      3930 non-null   object 
 14  status                3930 non-null   object 
 15  tagline              

In [47]:
# redefinindo o índice
final_df = final_df.reset_index()
indices = pd.Series(final_df.index, index=final_df['original_title'])

In [48]:
# convertendo o dataframe para csv
final_df.to_csv('final.csv')

In [ ]:
# baixando o arquivo
from google.colab import files
files.download('final.csv')